# 🎯 | Cora-For-Zava: Model Customization with Fine-Tuning

Welcome! This notebook will guide you through customizing an AI model using Supervised Fine-Tuning (SFT) to improve tone, style, and response consistency for your specific use case.

## 🛒 Our Zava Scenario

**Cora** is a customer service chatbot for **Zava** - a fictitious retailer of home improvement goods for DIY enthusiasts. While we've used few-shot examples and RAG to improve Cora's responses, these approaches increase prompt length (raising token costs and reducing available context window). Fine-tuning allows us to embed better tone and style directly into the model with shorter prompts.

## 🎯 What You'll Build

By the end of this notebook, you'll have:
- ✅ Prepared and validated training datasets for fine-tuning
- ✅ Analyzed token usage to optimize training efficiency
- ✅ Uploaded training data to Azure OpenAI for processing
- ✅ Submitted and monitored a fine-tuning job
- ✅ Deployed a custom fine-tuned model for testing
- ✅ Evaluated the improved tone and style consistency

## 💡 What You'll Learn

- How to prepare JSONL datasets for supervised fine-tuning
- How to validate token counts and optimize training data
- How to submit and monitor fine-tuning jobs in Azure OpenAI
- How to deploy and test fine-tuned models
- How fine-tuning reduces prompt length while improving consistency
- When to use fine-tuning vs. few-shot prompting vs. RAG

> **Note**: Fine-tuning customizes model behavior at the foundation level, allowing shorter prompts while maintaining quality and consistency.

Ready to customize your model? Let's get started! 🚀

---

## Step 1: Verify Environment Variables

The following environment variables should already be configured in your `.env` file from the earlier setup steps:

- **AZURE_OPENAI_API_KEY**: Your Azure OpenAI API key
- **AZURE_OPENAI_ENDPOINT**: Your Azure OpenAI service endpoint
- **AZURE_OPENAI_API_VERSION**: The API version to use (2025-02-01-preview for fine-tuning)
- **AZURE_SUBSCRIPTION_ID**: Your Azure subscription ID
- **AZURE_RESOURCE_GROUP**: Your Azure resource group name
- **AZURE_AI_PROJECT_NAME**: Your Azure AI Foundry project name

> **Important**: Fine-tuning requires specific API versions and model availability. Currently, `gpt-4o-2024-08-06` can be fine-tuned in Sweden Central and North Central US regions.

In [ ]:
import os

openai_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model_name = "gpt-4.1"
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2025-02-01-preview")

if not openai_key or not openai_endpoint:
    print("Error: Missing AZURE_OPENAI_KEY or AZURE_OPENAI_ENDPOINT environment variable.")

print("Using Model:", model_name)
print("Using API Version:", api_version)

## Step 2: Prepare Training Dataset

Fine-tuning requires carefully prepared training data in JSONL format. Each line contains a conversation example with the desired tone and style. Our dataset includes:

- **Training Set** (`31-basic_training.jsonl`): Examples for model learning
- **Validation Set** (`31-basic_validation.jsonl`): Examples for performance monitoring during training

> **Key Format Requirements**:
> - Each example must have a `messages` array with conversation turns
> - Messages include `role` (system/user/assistant) and `content`
> - Training examples should demonstrate the desired Zava tone: polite, factual, helpful

In [ ]:
# Identify Training and Validation datafiles


training_file = "./31-basic_training.jsonl" 
validation_file = "./31-basic_validation.jsonl"

In [ ]:
# Run preliminary checks

import json

# Load the training set
with open(training_file, 'r', encoding='utf-8') as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Number of examples in training set:", len(training_dataset))
print("First example in training set:")
for message in training_dataset[0]["messages"]:
    print(message)

# Load the validation set
with open(validation_file, 'r', encoding='utf-8') as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNumber of examples in validation set:", len(validation_dataset))
print("First example in validation set:")
for message in validation_dataset[0]["messages"]:
    print(message)

## Step 3: Assess Token Counts For Data

Token analysis is crucial for fine-tuning cost estimation and quality. We'll analyze:

- **Total Tokens**: Complete conversation length including system, user, and assistant messages
- **Assistant Tokens**: Only the model's responses (what we're training to improve)
- **Distribution Statistics**: Min/max, mean/median, and percentile analysis

> **Best Practices**:
> - Keep conversations focused and concise
> - Aim for consistent token lengths across examples
> - Monitor assistant token ratio for cost optimization

In [ ]:
# Validate token counts

import json
import tiktoken
import numpy as np

encoding = tiktoken.get_encoding("o200k_base") # default encoding for gpt-4o models. This requires the latest version of tiktoken to be installed.

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

files = [training_file, validation_file]

for file in files:
    print(f"Processing file: {file}")
    with open(file, 'r', encoding='utf-8') as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

    print_distribution(total_tokens, "total tokens")
    print_distribution(assistant_tokens, "assistant tokens")
    print('*' * 50)

## Step 4: Upload Fine-Tuning Data To Azure

Upload your prepared datasets to Azure OpenAI for processing. The uploaded files will be:

- **Stored securely** in your Azure OpenAI resource
- **Validated automatically** for format compliance
- **Accessible** for fine-tuning job creation
- **Visible** in Azure AI Foundry Portal under 'Data Files'

> **Security Note**: Data is encrypted at rest and in transit. Files are only accessible within your Azure OpenAI resource.

In [ ]:
# Create Azure OpenAI Client

import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version = os.getenv("AZURE_OPENAI_API_VERSION")
)

In [ ]:
# Upload the training and validation dataset files to Azure OpenAI with the SDK.
# Note: You can visit the Azure AI Foundry Portal - and look under your Azure AI Project's 'Data Files' tab to see the uploaded files.

training_response = client.files.create(
    file = open(training_file, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file = open(validation_file, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

## Step 5: Submit The Fine-Tuning Job

Create and submit a fine-tuning job with your uploaded datasets. Key parameters:

- **Base Model**: `gpt-4o-2024-08-06` (fine-tuning compatible version)
- **Training File**: Your uploaded training dataset
- **Validation File**: Your uploaded validation dataset  
- **Seed**: For reproducible results (optional but recommended)

> **Important**: Fine-tuning jobs typically take 10-30 minutes depending on dataset size. The job will automatically create training checkpoints and monitor validation loss.

In [ ]:
# Submit fine-tuning training job
# Add a delay so the cell runs for about 1 minute
import time
time.sleep(60)

# Note that the model you specify here must be one that can be fine-tuned.
# Currently gpt-4o can be fine-tuned only in Sweden Central and North Central US
# See: https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/fine-tuning?tabs=azure-openai&pivots=programming-language-python#prerequisites
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-4o-2024-08-06", # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters.
    seed = 105,  # seed parameter controls reproducibility of the fine-tuning job. If no seed is specified one will be generated automatically.
)

job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.
print("Job ID:", response.id)
print("Status:", response.id)
print(response.model_dump_json(indent=2))



---

## Step 6: Track Fine-Tuning Job Status

Monitor your fine-tuning job progress in real-time. The job progresses through these stages:

1. **Validating**: Checking data format and compatibility
2. **Running**: Active training with your dataset
3. **Succeeded**: Training completed successfully
4. **Failed**: Training encountered an error

> **Monitoring**: The cell will automatically refresh every 10 seconds until completion. You can also view progress in the Azure AI Foundry Portal.

In [ ]:
# Track training status

from IPython.display import clear_output
import time

start_time = time.time()

# Get the status of our fine-tuning job.
response = client.fine_tuning.jobs.retrieve(job_id)

status = response.status

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)

    response = client.fine_tuning.jobs.retrieve(job_id)
    print(response.model_dump_json(indent=2))
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = response.status
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Fine-tuning job {job_id} finished with status: {status}')

# List all fine-tuning jobs for this resource.
print('Checking other fine-tune jobs for this resource.')
response = client.fine_tuning.jobs.list()
print(f'Found {len(response.data)} fine-tune jobs.')

---

## Step 7: Review Fine-Tuning Events

Examine detailed training events and logs to understand the training process:

- **Training Progress**: Step-by-step training updates
- **Loss Metrics**: Training and validation loss evolution
- **Completion Status**: Final training results and model location

> **Debugging**: Events help troubleshoot any training issues and verify successful completion.

In [ ]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10)
print(response.model_dump_json(indent=2))

---

## Step 8: Review Fine-Tuning Checkpoints

Examine training checkpoints created during the fine-tuning process:

- **Checkpoint Analysis**: Model state at different training steps
- **Performance Metrics**: Validation loss at each checkpoint
- **Model Selection**: Identify the best performing checkpoint

> **Advanced Usage**: Checkpoints allow you to select optimal training stopping points and analyze training dynamics.

In [ ]:
response = client.fine_tuning.jobs.checkpoints.list(job_id)
print(response.model_dump_json(indent=2))

---

## Step 9: Retrieve Fine-Tuned Model Name

Get your completed fine-tuned model identifier for deployment:

- **Model ID**: Unique identifier for your custom model
- **Training Stats**: Final training metrics and completion details
- **Deployment Ready**: Model is ready for Azure deployment

> **Next Step**: Use this model ID to create a deployment in Azure AI Foundry Portal.

In [ ]:
# Retrieve fine_tuned_model name

response = client.fine_tuning.jobs.retrieve(job_id)

print(response.model_dump_json(indent=2))
fine_tuned_model = response.fine_tuned_model

---

## Step 10: Deploy Fine-Tuned Model For Testing

Deploy your fine-tuned model to test the improved tone and style:

1. **Azure AI Foundry Portal**: Navigate to your project's Model deployments
2. **Deploy Custom Model**: Select your fine-tuned model ID
3. **Developer Tier**: Use for testing (inference costs only, no hosting fees)
4. **Configure Deployment**: Set deployment name and resource allocation

> **Cost Optimization**: Developer tier allows testing without hosting costs. Upgrade to standard deployment for production use.

> Prompt 1: What kind of paint should I buy for my outdoor deck?

```txt
🪵 Deck protection options! Semi-Transparent Deck Stain at 38 enhances wood grain, or Deck & Fence Stain at 36 for UV protection?
```

> Prompt 2: I'm painting over rust - what spray paint should I use?

👍 Right choice! Rust Prevention Spray at $13 applies directly over rust with long-lasting protection. Primer recommendation?

## 🎯 Fine-Tuning Results & Insights

**Zava Tone Consistency Achieved!**

In both examples above, the fine-tuned model now consistently follows Zava's brand guidelines for "polite, factual, and helpful" responses:

### ✅ Consistent Structure
- **Emoji Opening**: Every response starts with a relevant emoji
- **Polite Acknowledgment**: First sentence acknowledges the customer's need
- **Factual Information**: Middle section provides specific product details and pricing
- **Helpful Follow-up**: Final sentence offers additional assistance

### 🚀 Key Benefits Achieved
- **Shorter Prompts**: No need for few-shot examples in every request
- **Lower Token Costs**: Reduced prompt length saves on API costs
- **Faster Processing**: Less context to process means faster responses
- **Consistent Quality**: Every response follows the trained pattern

### 📊 Performance Comparison
| Aspect | Before Fine-Tuning | After Fine-Tuning |
|--------|--------------------|--------------------|
| Prompt Length | ~800 tokens (with examples) | ~200 tokens (system prompt only) |
| Tone Consistency | Variable (depends on examples) | Consistent (embedded in model) |
| Response Time | Slower (longer prompt) | Faster (shorter prompt) |
| Cost per Request | Higher (more input tokens) | Lower (fewer input tokens) |

> **Best Practice**: Fine-tuning is most effective when you have consistent patterns you want the model to learn, rather than just providing factual knowledge (which is better handled by RAG).

## Step 11: Next Steps

You've successfully fine-tuned a model for better tone and style! Here are your next steps:

> **Key Insight**: Fine-tuning excels at embedding consistent patterns (like tone and style) while RAG excels at providing up-to-date factual information. Combining both creates powerful, cost-effective AI systems.

---

**Great work! You've mastered fine-tuning for tone and style customization.** 🎉